# QR factorization - a Numpy implementation

QR factorization factorizes a matrix **A** into an orthonormal matrix **Q** and a triangular matrix **R**.

$$\textbf{Q}\textbf{R}=\textbf{A}$$

<!-- TEASER_END -->

This factorization can be used to solve matrix equations or to solve the least squares problem. Numpy already has a QR function (np.linalg.qr), but this is a way to cement my own understanding of the concepts.

The matrix **Q** is created by orthonormalizing the original matrix **A**. To do this, first orthogonalize the columns of **A**, then divide each column-vector by it's norm.

In order to do this, it's necessary project one vector orthogonally to another, i.e. extract the part of one vector that is at a right angle to another. The projections of vector **b** parallel and orthogonal to vector **v** are given by:

$$\boldsymbol{b}=\boldsymbol{b}^{\parallel \boldsymbol{v}}+\boldsymbol{b}^{\bot \boldsymbol{v}}$$

The parallel projection takes two steps. First, find a value for the scalar sigma. Then, scale vector **v** by sigma.

$$\sigma =\dfrac{\langle \boldsymbol{b},\boldsymbol{v}\rangle}{\langle \boldsymbol{v}, \boldsymbol{v}\rangle}$$

$$\boldsymbol{b}^{\parallel \boldsymbol{v}}=\sigma \boldsymbol{v}$$

In [4]:
import numpy as np
import math

def project_parallel(b, v, epsilon= 1e-15):
    '''
    Projection of b along v
    
    Inputs:
        - b: 1-d array
        - v: 1-d array
        - epsilon: threshold for filling 0 for squared norms
    
    Output:
        - 1-d array: the projection of b parallel to v
    '''
    sigma = (np.dot(b,v)/np.dot(v,v)) if np.dot(v,v) > epsilon else 0
    return (sigma*v)

Now the orthogonal projection is simple: subtract the parallel component of **b** from the original vector **b**. This be done iteratively to orthogonalize **b** to a list of vectors.

In [5]:
def project_orthogonal(b, A):
    '''
    Project b orthogonal to row vectors of A

    Inputs:
        - b: 1-d array
        - A: 2-d array

    Output: the projection of b orthogonal to the row vectors of A
    '''
    for v in A:
        b = b - project_parallel(b, v)
    return b

A quick test to see it in action.

In [6]:
A = np.array([[1, 0, 0], [0, 1, 0]])
print (A, '\n')
b = np.array([1, 1, 1])
print (b)
project_orthogonal(b, A)

[[1 0 0]
 [0 1 0]] 

[1 1 1]


array([ 0.,  0.,  1.])

A set of vectors can now be orthogonalized by iterating through each one, projecting it orthogonally to an previously orthogonalized vectors. The first vector will be unchanged, the second vector will be projected orthogonally to vector 1, the third vector will be projected orthogonally to vector 1 and the orthogonalized vector 2, etc.

In [7]:
def orthogonalize(A):
    '''
    Orthogonalizes the row vectors of A.
    Row i of the output matrix is the projection of row i of the input
    matrix orthogonal to the space spanned by all previous rows in the 
    input matrix.
    
    Input: 2-d array
    
    Output: 2-d array of mutually orthogonal row vectors spanning the
            same space as the original row space'''
    
    orth_list = np.zeros(A.size)
    orth_list.shape = A.shape
    for i, v in enumerate(A):
        orth_list[i] = (project_orthogonal(v, orth_list))
    return orth_list

In [8]:
A = np.array([[8, -2, 2], [4, 2, 4]])
orthogonalize(A)

array([[ 8., -2.,  2.],
       [ 0.,  3.,  3.]])

Now that the vectors are mutually orthogonal, they can be normalized by scaling each by 1/norm. The norm of a vector is given by:
$$\left\| \boldsymbol{v} \right\|=\sqrt {\langle \boldsymbol{v}, \boldsymbol{v}\rangle}$$

In [9]:
def orthonormalize(A):
    '''
    Orthonormalizes the row vectors of A
    
    Input: 2-d array
    
    Output: 2-d array of orthonormalized vectors
    '''
    return np.stack([v/math.sqrt(sum(v**2)) for v in orthogonalize(A)])

In [10]:
orthonormalize(A)

array([[ 0.94280904, -0.23570226,  0.23570226],
       [ 0.        ,  0.70710678,  0.70710678]])

The groundwork has been layed to calculate **Q**, but there's still the question of how to calculate **R**. It turns out that there's a pretty simple way to do this. **Q** is an orthonormal matrix, and thus has the property:
$$\textbf{Q}^{-1}=\textbf{Q}^{T}$$

therefore:

$$\textbf{R}=\textbf{Q}^{T}\textbf{A}$$

In [21]:
def QR_factorize(A):
    '''
    Factorizes A into orthonormal matrix Q and triangular matrix R
    
    Input: a 2-d array with linearly independent columns
    
    Outputs:
        Q = a matrix of orthonormal colum vectors that span the same column
            space as the input
        R = a triangualr matrix of vectors such that Q*R = A
    '''
    Q = orthonormalize(A.T)
    R = np.dot(Q, A)
    return Q.T, R

In [30]:
A = np.array([[4, 8, 10], [3, 9, 1], [1, -5, -1], [2, -5, 5]])
Q, R = QR_factorize(A)

print('Original matrix A:')
print(A, '\n')
print('Q:')
print(Q, '\n')
print('R:')
print(R, '\n')
print('Q*R')
print(np.dot(Q, R), '\n')


Original matrix A:
[[ 4  8 10]
 [ 3  9  1]
 [ 1 -5 -1]
 [ 2 -5  5]] 

Q:
[[ 0.73029674  0.18677078  0.5275409 ]
 [ 0.54772256  0.4027245  -0.6531217 ]
 [ 0.18257419 -0.56614893 -0.51230873]
 [ 0.36514837 -0.69455384  0.18075511]] 

R:
[[  5.47722558e+00   8.03326418e+00   9.49385766e+00]
 [  2.22044605e-16   1.14222006e+01  -6.36187974e-01]
 [ -2.22044605e-16  -4.44089210e-16   6.03837160e+00]] 

Q*R
[[ 4.  8. 10.]
 [ 3.  9.  1.]
 [ 1. -5. -1.]
 [ 2. -5.  5.]] 



There it is: matrix **A** factorized then recomposed. **Q** is orthonormal and **R** is triangular. Compare the result with the function from Numpy.

In [31]:
Q, R = np.linalg.qr(A)
print('Q:')
print(Q, '\n')
print('R:')
print(R, '\n')
print('Q*R')
print(np.dot(Q, R), '\n')

Q:
[[-0.73029674 -0.18677078  0.5275409 ]
 [-0.54772256 -0.4027245  -0.6531217 ]
 [-0.18257419  0.56614893 -0.51230873]
 [-0.36514837  0.69455384  0.18075511]] 

R:
[[ -5.47722558  -8.03326418  -9.49385766]
 [  0.         -11.42220061   0.63618797]
 [  0.           0.           6.0383716 ]] 

Q*R
[[  4.   8.  10.]
 [  3.   9.   1.]
 [  1.  -5.  -1.]
 [  2.  -5.   5.]] 



The **Q**s and **R**s are slightly different, but both are correct factorizations. What about speed?

In [26]:
A = np.random.rand(1000, 1000)

In [27]:
%%time
Q, R = QR_factorize(A)

CPU times: user 12.8 s, sys: 64.6 ms, total: 12.9 s
Wall time: 16.7 s


In [28]:
%%time
Q, R = np.linalg.qr(A)

CPU times: user 381 ms, sys: 41.9 ms, total: 423 ms
Wall time: 337 ms


My version is much, much slower, and I would use numpy in practice, but now I have a better understanding of how QR factorization works.